<a href="https://colab.research.google.com/github/ihatestudysomuch/pytorch/blob/main/Pytorch_9%EC%9E%A5_Data_Dataset_DataLoader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data - train, validation, test


1.   딥러닝에 사용되는 data는 train, validation, test 일반적으로 이 세 가지로 나뉘어져있다.
  - train data는 학습에 사용되는 data로 가중치 W, 바이어스 b를 최적화 하기 위해 사용된다.
  - validataion data는 epoch마다 과적합을 확인하기 위해 사용된다. 이해하기 어렵다.. 풀어서 쓰면 현재 model이 얼마나 정확하고 손실을 낮출 수 있는지 검증하는 data이다.
  - test data는 학습을 모두 마친 후, 평가하거나 임의의 입력에 대한 결과를 예측하기 위해 사용되는 data이다.
1.  이때까지 학습데이터를 통째로 모델에 넣어 예측값과 정답값을 통째로 비교했지만, 데이터가 매우 클 경우 컴퓨터 자원과 시간, 계산의 한계가 발생하므로 batch라는 한 번에 입력되는 데이터 묶음 단위를 정하고 전체 데이터를 이러한 batch size로 나눠서 순차적으로 모델에 입력하는 것이 일반적이다.




Dataset - DataLoader



1.   Data 부분에서 말했듯, 입력과 정답을 저장하고 data를 batch size 간격으로 나눈 후에 batch size 만큼 입력 데이터와 정답 데이터를 넣어주는 기능이 필요한데 이 기능을 pytorch에서 Dataset class / DataLoader class 이다.
2.   Dataloader - CustomDataset을 각각 생성하면 batch data를 제공받을 수 있다.



In [38]:
import torch

x_train = torch.Tensor([1,2,3,4,5,6]).view(6,1)
y_train = torch.Tensor([3,4,5,6,7,8]).view(6,1)

In [39]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset): # Dataset을 상속받는 CustomDataset 정의
  # CustomDataset (우리가 직접 만든 Dataset)에서는 __init__, __getitem__, __len__을 정의해야만 한다.
  # __init__은 입력 데이터와 정답 데이터를 저장하는 기능을 한다.
  def __init__(self, x_train, y_train):
    self.x_train = x_train
    self.y_train = y_train

  # index에 해당하는 입력 데이터, 정답 데이터를 return하는 기능을 한다.
  def __getitem__(self, index):
    return self.x_train[index], self.y_train[index]

  # 현재 data size를 return하는 기능을 한다.
  def __len__(self):
    return self.x_train.shape[0]

In [40]:
# Dataset 정의
dataset = CustomDataset(x_train, y_train)

# DataLoader 정의, Dataset, batch size, shuffle 여부를 기본으로 정의 가능하다.
train_loader = DataLoader(dataset=dataset, batch_size = 3, shuffle=True)

In [41]:
from torch import nn

# 모델 생성
class MyLinearRegressionModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.linear_stack = nn.Sequential(
            nn.Linear(1, 1) # 입력 데이터 1, 출력 데이터 1
        )

    # 피드 포워드
    def forward(self, data):
        prediction = self.linear_stack(data)

        return prediction


In [42]:
model = MyLinearRegressionModel()

In [43]:
loss_function = nn.MSELoss() # 손실함수, 평균제곱의 오차

optimizer = torch.optim.SGD(model.parameters(), lr=1e-2) # 옵티마이저 경사하강법

In [44]:
for epoch in range(2):

    for idx, batch_data in enumerate(train_loader):

        x_train_batch, y_train_batch = batch_data

        output_batch = model(x_train_batch)

        print('==============================================')
        print('epoch =', epoch+1, ', batch_idx =', idx+1, ',',
              len(x_train_batch), len(y_train_batch), len(output_batch))
        print('==============================================')

        # 역전파
        loss = loss_function(output_batch, y_train_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

epoch = 1 , batch_idx = 1 , 3 3 3
epoch = 1 , batch_idx = 2 , 3 3 3
epoch = 2 , batch_idx = 1 , 3 3 3
epoch = 2 , batch_idx = 2 , 3 3 3
